In [7]:
#load all the functions from other scripts
%run IP/IP_methods.ipynb #Image processing script



In [8]:
#different path usefull locations
Path_to_MAP = 'output'

In [12]:
(x,y), orientation = pipeline_IP(plot = False)